In [1]:
import json
import numpy as np
import os
import paramiko
import sklearn
import subprocess

print(np.__version__)
print(sklearn.__version__)

1.18.5
0.24.1


## Multivariate Reproducibility

This notebook showcases the reproducibility issues with sampling from a multivariate normal distribution in NumPy. It uses the following [GitHub issue](https://github.com/numpy/numpy/issues/2435) as an example.

In [2]:
%%writefile example_normal.py

import numpy as np
import sys

random_option = sys.argv[1]

d = 10
alpha = 1 / d**0.5
mu = np.ones(d)
R = alpha * np.ones((d, d)) + (1 - alpha) * np.eye(d)
ls = []
for _ in range(100):
    if random_option == 'randomstate':
        rng = np.random.RandomState(seed=587482)
        ls.append(rng.multivariate_normal(mu, R, 1))
    elif random_option == 'svd':
        rng = np.random.default_rng(587482)
        ls.append(rng.multivariate_normal(mu, R, 1))
    elif random_option == 'cholesky':
        rng = np.random.default_rng(587482)
        ls.append(rng.multivariate_normal(mu, R, 1, method='cholesky'))
vals = np.unique(np.vstack(ls), axis=0).squeeze()
assert len(vals.shape) == 1
print(vals.tolist())

Overwriting example_normal.py


We test 4 environments, two on Mac and two on Linux, both looking at OpenBLAS vs MKL. Note that you need to create a `ssh_config.json` file, and you can find an example `example_ssh_config.json`.

In [3]:
with open('ssh_config.json') as fp:
    conf = json.load(fp)

local_conf = {'host_name': 'localhost'}

envs = [
    ('mac_openblas', local_conf, os.path.expanduser('~/Tools/miniconda3/envs/reprod/bin/python')),
    ('mac_mkl', local_conf, os.path.expanduser('~/Tools/miniconda3/envs/reprod2/bin/python')),
    ('linux_openblas', conf, '$HOME/Tools/miniconda3/envs/reprod/bin/python'),
    ('linux_mkl', conf, '$HOME/Tools/miniconda3/envs/reprod2/bin/python'),
]

def test_envs(env_lis, script, option):
    all_vals = []
    for env_name, conf, python_exec in env_lis:
        if conf['host_name'] == 'localhost':
            vals = eval(subprocess.run([python_exec, script, option], capture_output=True).stdout.decode('utf-8').strip())
            vals = np.array(vals)
            all_vals.append(vals)
        else:
            with paramiko.SSHClient() as ssh:
                ssh.load_system_host_keys()
                ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
                ssh.connect(hostname=conf['host_name'], username=conf['username'], key_filename=os.path.expanduser(conf['key_filename']))
                with ssh.open_sftp() as ftp:
                    ftp.put(script, f'/tmp/{script}')

                _, stdout, stderr = ssh.exec_command(f'{python_exec} /tmp/{script} {option}', get_pty=True)

                # Log the stdout as it comes
                vals = eval('\n'.join(stdout.readlines()).strip())
                all_vals.append(np.array(vals))
    return np.unique([np.floor(v * 1e7) / 1e7 for v in all_vals], axis=0)

### Scenario 1

Using the old `RandomState` class, we sample from the multivariate normal:

In [4]:
test_envs(envs, 'example_normal.py', 'randomstate')

array([[ 0.097969 ,  0.4400853,  0.2609484,  0.8460852,  0.7329605,
         1.3780882, -0.7805136, -0.200187 , -0.8869034, -0.9147636],
       [ 0.097969 ,  0.8510258, -1.2944923, -1.1840146,  0.485418 ,
        -0.239079 ,  0.7711746,  0.7755532,  0.0871302,  0.6230842]])

It is obvious that there are divergent values.

### Scenario 2
Using the new `Generator` class, we sample from the multivariate normal with default arguments:

In [5]:
test_envs(envs, 'example_normal.py', 'svd')

array([[ 1.5067828,  1.7385225, -0.5997608,  0.530395 ,  0.0854511,
         1.8629863,  1.8233736,  2.8039635,  1.5785772,  1.2154195],
       [ 1.5067828,  1.8202132,  1.6301911,  1.1709111,  1.9955093,
         2.3661913,  1.8158426,  0.596513 , -1.0887922,  0.7323486]])

Again, it is obvious that there are divergent values.

### Scenario 3
Using the new `Generator` class with the Cholesky decomposition method, we sample from the multivariate normal:

In [6]:
test_envs(envs, 'example_normal.py', 'cholesky')

array([[ 0.5895108,  2.7756236, -1.0727797, -0.072917 ,  0.8692535,
         1.2042482,  1.3048694,  0.9933494,  0.4586954,  1.102799 ]])

Cholesky decomposition gives us deterministic samples across all environments.